In [ ]:
from insta_mainfile import *

In [ ]:
#URL만 크롤링 하는 부분
#바꿀부분: 아이디, 비밀번호, chromedriver 경로

In [ ]:
userId = "아이디"
userPw = "비밀번호"

instaPage = page("/Users/gyeonga/Desktop/chromedriver");
instaPage.open('https://www.instagram.com/accounts/login/?next=https://www.instagram.com/&source=mobile_nav');
instaPage.login(userId, userPw)
instaPage.search("인스타푸드")

start = time.localtime()
print("start time: ", start)
driver = instaPage.getDriver()
post_list = list_page(driver)
post_list.crawling(400)
end = time.localtime()
print("end time: ", end)
print()

url_list = post_list.get_url_list()
print("url_list: ")
print(url_list)

In [ ]:
print(len(url_list))

In [ ]:
url_list

-----------------------

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
import requests
from multiprocessing import Pool
import multiprocessing
from urllib.request import urlopen, Request
from urllib.parse import quote_plus
import json

In [ ]:
#URL 한개로 실험, 바꿀 부분 없음

In [ ]:
#new_url_list = ['/p/CLy657ulL5q/']
new_url_list = url_list[0:300]

In [ ]:
new_url_list

In [ ]:
datas = {"날짜":[], "위치":[], "본문":[], "댓글수":[], "좋아요수":[], "해시태그":[], "댓글":[]}

In [ ]:
def get_infos(url):

    full_url = "https://www.instagram.com" + url
    req = Request(full_url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, "html.parser", from_encoding="euc-kr")
    print(soup)

    try:
        js = json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))
    except:
        js = ""
    try:
        date = js["uploadDate"]
    except: 
        date = "-"
    try:
        location = js["contentLocation"]["address"]["streetAddress"]
    except:
        location = "-"
    try:
        body = soup.find("meta", {"property":"og:title"}).get("content")
        body = js["caption"]
    except:
        body = "-"
    
    try:
        numComments = js["commentCount"]
    except:
        numComments = "-"
    try:
        numLikes = js["interactionStatistic"]["userInteractionCount"]
    except:
        numLikes = "-"
    
    try:
        comment = js["comment"]
        comment_list = []
        for c in comment:
            text = c["text"]
            person = c["author"]["alternateName"]
            profile = c["author"]["mainEntityofPage"]["@id"]
        
            person_dict = {"text": text, "person": person, "profile":profile}
            comment_list.append(person_dict)
    except:
        comment = ""
        comment_list = ["-"]
    
    try:
        hashtag = soup.find_all("meta", {"property": "instapp:hashtags"})
        hash_list = []
        for tag in hashtag:
            hash_list.append(tag.get("content"))
        videotag = soup.find_all("meta", {"property": "video:tag"})
        for tag in videotag:
            hash_list.append(tag.get("content"))
    except:
        hash_tag = ["-"]

    datas["날짜"].append(date)
    datas["위치"].append(location)
    datas["본문"].append(body)
    datas["좋아요수"].append(numLikes)
    datas["댓글수"].append(numComments)
    datas["해시태그"].append(hash_list)
    datas["댓글"].append(comment_list)

In [ ]:
start = time.localtime()
print("start time: ", start)
for (idx, url) in enumerate(new_url_list):
    print(idx)
    get_infos(url)
end = time.localtime()
print("end time: ", end)
print()

In [ ]:
df = pd.DataFrame(datas)